# TITAN Tutorial

---

<img align="right" width="300" height="300" src="../images/hail_case_tracks.png">

This interactive tutorial takes you through the steps of how to run the Thunderstorm Identification, Tracking, Analysis and Nowcasting (TITAN) suite. TITAN was originally designed as an algorithm to objectively identify and track thunderstorms from weather radar data for a weather modification experiment in South Africa in the 1980s. Now, Titan includes forecasting, storm analysis, and climatological analysis. TITAN now refers to the larger system in which the original application is one component.

Titan is described in more detail in [Dixon and Wiener (1993)](https://doi.org/10.1175/1520-0426(1993)010%3C0785:TTITAA%3E2.0.CO;2).

---

## Tutorial Overview
### 1. Setup

#### Download raw data and prepare parameter files

Raw data files that are provided:
* HDF 5 files from ECCC

Parameter files :

* Titan params

### 2. Prepare data for analysis

* Convert Odim files to CfRadial
    * RadxConvert
* ????

### 3. Run Titan

* Calculate KDP, run PID algorithm, estimate three-dimensional precipitation rates
    * RadxRate

### 4. Plot Results

* Visualize results of Titan analysis

### 5. Note on task cells

This notebook uses two colored cells to indicate tasks.

<div class="alert alert-block alert-info"> <b>File Task: modify parameters in text files.</b> 

These text blocks help the user modify the parameter files or other functions in *external* text files.

</div>

<div class="alert alert-block alert-warning"> <b>Cell Task: run a command in Jupyter notebook cell.</b> 

These text blocks instruct the users to run a command *in* a cell within the Jupyter notebook. If you prefer, you are welcome to copy the commands (minus the ! symbol) into a terminal window.

</div>

---

# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

## Set up directories

We need to set up the required data directories. The raw radar data will be grabbed from the S3 buckt. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
## make a directory for all the data files in the echo tutorial (raw and analysis)
!rm -rf ./data/titan
!mkdir -p ./data/titan

## make subdirectory within data for the converted data
!rm -rf ./data/titan/convert
!mkdir ./data/titan/convert

## make subdirectory within data for output data
!rm -rf ./data/titan/output
!mkdir ./data/titan/output


# 2. Prepare data for analysis
## 2.1 Convert Nexrad Level II to CfRadial files

We can examine the raw radar file using RadxPrint. RadxPrint reads CfRadial and other raw radar formats and prints out radar metadata, variable names, and sweep information. For example, we can look at the variables using the following command, piping the output into the bash head command.

In [ ]:
# print out the first 50 lines of RadxPrint output
!RadxPrint -f ./data/raw/2024080600_00_ODIMH5_PVOL6S_VOL_CASSM.h5 | head -50


***... add in file streaming code from basics notebook?????? ...***

We convert the raw to CfRadial using RadxConvert so the other applications can read the files. Note that in the vast majority of cases, RadxConvert can be run without a parameter file, including in this case. However, there are instances where a parameter file is useful.

Here, the *-f* flag provides the path to the raw radar file and the *-outdir* flag tells RadxConvert where the CfRadial file will be written. 

In [ ]:
!RadxConvert -f ./data/titan/raw/*.nc -outdir ./data/convert/cfradial


## 2.2 Visually Inspect Data

First, let's look at a basic plot of the raw data using Py-ART.

In [ ]:
# Read CfRadial file into radar object
inDir_raw = "./data/titan/cfradial/20240805/"
file_raw = "cfrad.20240805_235403.000_to_20240805_235938.000_unknown_SUR.nc"
chill_raw = pyart.io.read_cfradial(inDir_raw+file_raw)

# Plot raw data

swp = 14

displayRaw = pyart.graph.RadarDisplay(chill_raw)
figRaw = plt.figure(1, (12, 5))

# DBZ (input)

axDbz = figRaw.add_subplot(121)
displayRaw.plot_ppi('DBZH', swp, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ", cmap='ChaseSpectral')
displayRaw.plot_range_rings([50, 100, 150, 200, 250])
displayRaw.plot_cross_hair(250.)
displayRaw.set_limits(xlim=(-300,300),ylim=(-300,300))

# ZDR

axZdr = figRaw.add_subplot(122)
displayRaw.plot_ppi('ZDR', swp, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRaw.plot_range_rings([50, 100, 150, 200, 250])
displayRaw.plot_cross_hair(250.)
displayRaw.set_limits(xlim=(-300,300),ylim=(-300,300))

## 2.3 Other program placeholder (QC?)
... other processing placeholder

## 2.4 Other program placeholder (sounding needed?)

... other processing placeholder


## 2.5 Other program placeholder

... other processing placeholder


# 3. Setup and run Titan

## Titan Overview

... Summarize prior processing and what Titan will do now ...

Running Titan requires # main parameter file on the command line. That parameter file contains ... .

Titan requires ... . 

<div class="alert alert-block alert-warning"> <b>Cell Task: Check command line options of Titan</b> 
    <br>
    LROSE applications often let you link to the input files and set the output directory on the command line. Let's check that those options exist (e.g., -f and -outdir). Type the following command into the open cell below and run it!
    <br>
    <code lang="bash">!Titan -h</code>
</div>

## 3.1 Generate the parameter file
First, we need to generate the parameter file! LROSE applications use the <code lang="bash">-print_params</code> flag to generate parameter files with the defaults. We just need to specify the location and name of the parameter file. 

Titan requires # parameter files to run: 
* a main parameter file that sets variable names and file paths

<div class="alert alert-block alert-warning"> <b>Cell Task: Create the Titan parameter file.</b> 
<br>
Type the following commands into the open cell below and run it!
<br>
<code lang="bash">!Titan -print_params > ./params/Titan_params</code>
</div>

## 3.2 Fill out the Titan parameter file

Now we will step through and edit the parameter files.

**Titan Parameter file (Titan_params)**

First, we'll start with the parameters associated with the necessary file paths. Since we determined earlier that we can link to the input files and output directory on the command line, we don't need to modify them here, but we've listed the line numbers for reference.

| Parameter | Description |
| --- | --- |
|input_dir|Path to input data, if not already specified on the command line.|


<div class="alert alert-block alert-info"> <b>File Task: ... .</b> 

Placeholder for tasks in the parameter file.

</div>

---

Next, we'll ...

<div class="alert alert-block alert-warning"> <b>Cell Task: ... .</b> 
    
Example Titan task that should be run in the cell

</div>



---

**Now we can run Titan!**

Here, *-params* provides the link to the main RadxRate parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where Titan should write the final files.

In [ ]:
!Titan -params ./params/Titan_params -f -outdir ./data/titan/


# 4. Plot Output